In [ ]:
# Uncomment the below command if you don't have openpyxl
#!pip install openpyxl 

# TODO
### PUT FUNCTIONS TO PYTHON SCRIPT, OUTPUT SOME EXAMPLE IN NOTEBOOK AND ADD DOCSTRING

In [1]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt

from collections import OrderedDict

### Battery Class

In [2]:
###################################
# A Class to store battery functionaly such as revenue, charge and discharge
# periods, charge and discharge spot prices, charge and discharge market dispatch
#
# NOTE : IF THERE IS ANY CHANGE IN CALCULATION, CHANGE CALCULATION MARKED BY (BATTERY CALCULATION).
#
# Consists of:
#      - charge_period : period when it should charge.
#      - discharge_period : period when it should discharge.
#      - charge_price : spot price given charging period.
#      - discharge_price : spot price given discharging period.
#      - charge_market_dispatch : set amount of market dispatch given charging period.
#      - discharge_market_dispatch : set amount of market dispatch given discharging period.
# 
# Functions:
#      - ComputeRevenue: To calculate revenue given discharge and charge period pairs.
#      - Setting : Set all the battery functionalities. (BATTERY CALCULATION)
#      - FirstOptimisation : Ensure that energy are not wasted or not used. (BATTERY CALCULATION)
#      - SecondOptimisation : Ensure that to always charge the highest price less and discharge
#                             highest price more. (BATTERY CALCULATION)
#
# Created by: Gilbert
###################################
class Battery:
    # Battery Specifications
    mlf = 0.991                 # Marginal Loss Factor
    battery_capacity = 580      # Battery Capacity
    battery_power = 300         # Battery Power
    charge_efficiency = 0.9     # Charge Efficiency
    discharge_efficiency = 0.9  # Discharge Efficiency
        
    def __init__(self, charge_period, charge_spot_price,
                 discharge_period, discharge_spot_price): 
        # Charge and Discharge Period
        self.charge_period = charge_period
        self.discharge_period = discharge_period
        
        # Spot Price during Charge and Discharge
        self.charge_price = charge_spot_price
        self.discharge_price = discharge_spot_price
    
    ########################################################################################    
                
    def Revenue(self):
        if self.charge_market_dispatch == [] or self.discharge_market_dispatch == []:
            return 0
        
        # Spot Prices
        charge_sp = np.array(self.charge_price)[:, 1]
        discharge_sp = np.array(self.discharge_price)[:, 1]
        
        # Market Dispatches
        charge_md = np.array(self.charge_market_dispatch).T
        discharge_md = np.array(self.discharge_market_dispatch).T
        
        # Revenues
        charge_revenue = (charge_sp @ charge_md) * (1 / self.mlf)
        discharge_revenue = (discharge_sp @ discharge_md) * (self.mlf)
        
        return discharge_revenue + charge_revenue
    
    ########################################################################################
    
    def Setting(self):
        
        OPENING = 0
        CLOSING = 1
        
        battery_power = self.battery_power
        battery_cap = self.battery_capacity
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        self.charge_raw_power = ['' for i in range(len_charge)]
        self.discharge_raw_power = ['' for i in range(len_discharge)]
        
        self.charge_market_dispatch = ['' for i in range(len_charge)]
        self.discharge_market_dispatch = ['' for i in range(len_discharge)]
        
        self.charge_capacity = [[0, 0] for i in range(len_charge)]
        self.discharge_capacity = [[0, 0] for i in range(len_discharge)]
        
        # CHARGE PERIOD --------------------------------------------------------------------
        for t in range(len_charge):
            # RAW_POWER[t] = -MIN(BATTERY_POWER, (BATTERY_CAPACITY - OPENING_CAPACITY[t]) / CHARGE_EFFICIENCY * 2)
            self.charge_raw_power[t] = -min(battery_power, 
                                           (battery_cap - self.charge_capacity[t][OPENING]) / 
                                            self.charge_efficiency * 2)

            # MARKET_DISPATCH[t] = RAW_POWER / 2
            self.charge_market_dispatch[t] = self.charge_raw_power[t] / 2

            # CLOSING_CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] - 
            #                        MARKET_DISPATCH[t] * CHARGE_EFFICIENCY, BATTERY_CAPACITY))
            self.charge_capacity[t][CLOSING] = max(0, min(self.charge_capacity[t][OPENING] - 
                                                        self.charge_market_dispatch[t] * self.charge_efficiency, 
                                                        battery_cap))

            # Ensuring that it doesn't exceeds array len limit
            if t + 1 < len_charge:
                self.charge_capacity[t + 1][OPENING] = self.charge_capacity[t][CLOSING]

        # DISCHARGE PERIOD -----------------------------------------------------------------

        # Set DISCHARGE CAPACITY AT t = 0 to be the the last t of CHARGING CAPACITY
        self.discharge_capacity[0][OPENING] = self.charge_capacity[-1][CLOSING]

        for t in range(len_discharge):
            # RAW_POWER[t] = MIN(BATTERY_POWER, OPENING_CAPACITY[t] * 2)
            self.discharge_raw_power[t] = min(battery_power, self.discharge_capacity[t][OPENING] * 2)

            # MARKET_DISPATCH[t] = RAW_POWER[t] / 2 * DISCHARGE EFFICIENCY
            self.discharge_market_dispatch[t] = self.discharge_raw_power[t] / 2 * self.discharge_efficiency

            # CLOSING CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] -
            #                        MARKET_DISPATCH[t] * (1/DISCHARGE_EFFICIENCY), BATTERY_CAPACITY))
            self.discharge_capacity[t][CLOSING] = max(0, min(self.discharge_capacity[t][OPENING] -
                                                            self.discharge_market_dispatch[t] * (1 / self.discharge_efficiency),
                                                            battery_cap))
            # Ensuring that it doesn't exceeds array len limit
            if t + 1 < len_discharge:
                self.discharge_capacity[t + 1][OPENING] = self.discharge_capacity[t][CLOSING]
    
    ########################################################################################
        
    def FirstOptimisation(self):
        OPENING = 0
        CLOSING = 1
        
        battery_power = self.battery_power
        battery_cap = self.battery_capacity
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        MAX_CHARGE_PERIOD = 5
        MAX_DISCHARGE_PERIOD = 4
        
        if (len_charge >= MAX_CHARGE_PERIOD and len_discharge >= MAX_DISCHARGE_PERIOD) or (len_charge == len_discharge):
            #md = 117
            #self.discharge_raw_power[-1] = min(md * 2 / self.discharge_efficiency, self.discharge_raw_power[-1])
            #self.discharge_market_dispatch[-1] = self.discharge_raw_power[-1] / 2 * self.discharge_efficiency
            return
        elif (len_charge - len_discharge == 1):
            self.discharge_capacity[-1][CLOSING] = 0
            # OPENING_CAPACITY[-1] = MARKET_DISPATCH / DISCHARGE_EFFICIENCY
            self.discharge_capacity[-1][OPENING] = self.discharge_market_dispatch[-1] / self.discharge_efficiency
            for t in range(1, len_discharge):
                # CLOSING_CAPACITY[-t - 1] = OPENING_CAPACITY[-t]
                self.discharge_capacity[-t - 1][CLOSING] = self.discharge_capacity[-t][OPENING]
                # OPENING_CAPACITY[-t - 1] = CLOSING_CAPACITY[-t - 1] + MARKET_DISPATCH[-t - 1] / DISCHARGE_EFFICIENCY
                self.discharge_capacity[-t - 1][OPENING] = self.discharge_capacity[-t - 1][CLOSING] + self.discharge_market_dispatch[-t - 1] / self.discharge_efficiency
            
            # CLOSING_CAPACITY[-1] = OPENING_CAPACITY[0]
            self.charge_capacity[-1][CLOSING] = self.discharge_capacity[0][OPENING]
            # MARKET_DISPATCH[-1] = -(CLOSING_CAPACITY[-1] - OPENING_CAPACITY[-1]) / CHARGE_EFFICIENCY
            self.charge_market_dispatch[-1] = -(self.charge_capacity[-1][CLOSING] - self.charge_capacity[-1][OPENING]) / self.charge_efficiency
            # RAW_POWER[-1] = MARKET_DISPATCH[-1] * 2
            self.charge_raw_power[-1] = self.charge_market_dispatch[-1] * 2
            
    ########################################################################################

    def SecondOptimisation(self):
        OPENING = 0
        CLOSING = 1
        
        battery_power = self.battery_power
        battery_cap = self.battery_capacity
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        MAX_CHARGE_PERIOD = 5
        MAX_DISCHARGE_PERIOD = 4
        
        # CHARGE PERIOD --------------------------------------------------------------------
        highest_price_index = np.array(self.charge_period).argmax(axis=0)[0]
        lowest_charge_index = self.charge_market_dispatch.index(max(self.charge_market_dispatch))    
        
        # IF THE HIGHEST CHARGING PRICE DOESN'T HAVE THE LOWEST CHARGING RATE, SWAP!
        if (highest_price_index != lowest_charge_index):
            tmp = self.charge_market_dispatch[highest_price_index]
            self.charge_market_dispatch[highest_price_index] = self.charge_market_dispatch[lowest_charge_index]
            self.charge_market_dispatch[lowest_charge_index] = tmp
            # SET THE UPDATED BATTERY SETTINGS.
            for t in range(len_charge):
                # RAW_POWER[t] = MARKET_DISPATCH[t] * 2
                self.charge_raw_power[t] = self.charge_market_dispatch[t] * 2

                # CLOSING_CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] - 
                #                        MARKET_DISPATCH[t] * CHARGE_EFFICIENCY, BATTERY_CAPACITY))
                self.charge_capacity[t][CLOSING] = max(0, min(self.charge_capacity[t][OPENING] - 
                                                            self.charge_market_dispatch[t] * self.charge_efficiency, 
                                                            battery_cap))
                # Ensuring that it doesn't exceeds array len limit
                if t + 1 < len_charge:
                    self.charge_capacity[t + 1][OPENING] = self.charge_capacity[t][CLOSING]

        # DISCHARGE PERIOD -----------------------------------------------------------------
        lowest_price_index = np.array(self.discharge_period).argmax(axis=0)[0] 
        min_discharge_index = self.discharge_market_dispatch.index(min(self.discharge_market_dispatch))   
        
        # IF THE LOWEST DISCHARGING PRICE DOESN'T HAVE THE LOWEST DISCHARGING RATE, SWAP!
        if (lowest_price_index != min_discharge_index):
            tmp = self.discharge_market_dispatch[lowest_price_index]
            self.discharge_market_dispatch[lowest_price_index] = self.discharge_market_dispatch[min_discharge_index]
            self.discharge_market_dispatch[min_discharge_index] = tmp
            # SET THE UPDATED BATTERY SETTINGS.
            for t in range(len_discharge):
                # RAW_POWER[t] = MARKET_DISPATCH[t] * 2 / DISCHARGE_EFFICIENCY
                self.discharge_raw_power[t] = self.discharge_market_dispatch[t] * 2 / self.discharge_efficiency

                # CLOSING_CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] - 
                #                        MARKET_DISPATCH[t] * CHARGE_EFFICIENCY, BATTERY_CAPACITY))
                self.discharge_capacity[t][CLOSING] = max(0, min(self.discharge_capacity[t][OPENING] - 
                                                            self.discharge_market_dispatch[t] / self.discharge_efficiency, 
                                                            battery_cap))
                # Ensuring that it doesn't exceeds array len limit
                if t + 1 < len_discharge:
                    self.discharge_capacity[t + 1][OPENING] = self.discharge_capacity[t][CLOSING]
        

### Helper function for battery optimisation

In [29]:
EMPTY = ' '
###################################
# A function to get spot prices based on selected regions.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - selected_periods : selected period for charging or discharging.
#      - region : the targeted region, default has been set to 'VIC' for mandatory task.
#
# Return:
#      - List of spot prices given period
#
# Created by: Gilbert
###################################
def GetSpotPrice(data, selected_periods = False, region = 'VIC'):
    if region == 'VIC':
        spot_price = data['Regions VIC Trading Price ($/MWh)']
    elif region == 'NSW':
        spot_price = data['Regions NSW Trading Price ($/MWh)']
    elif region == 'SA':
        spot_price = data['Regions SA Trading Price ($/MWh)']
    elif region == 'TAS':
        spot_price = data['Regions TAS Trading Price ($/MWh)']
    
    spot_price = np.array(spot_price)
    
    # Find the spot prices from selected region. Periods are
    # index + 1, therefore to use the index we need to subtract
    # it by 1.
    if (selected_periods):
        retrieved_prices = []
        for period in selected_periods:
            # append(minimum or maximum ranking, spot_price[index])
            retrieved_prices.append((period[1], spot_price[period[1] - 1]))
        return retrieved_prices
    else:
        return spot_price

########################################################################################

###################################
# A function to find minimum and maximum point rank given threshold.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - region : the targeted region, default has been set to 'VIC' for mandatory task.
#      - buy_threshold : maximum number of buying point, default has been set to optimise Checkpoint 3.
#      - sell_threshold : maximum number of selling point, default has been set to optimise Checkpoint 3.
#
# Return:
#      - List of selected minimum point, list of selected maximum point
#
# Efficiency: O(3N + NLogN) = O(NLogN)
#
# Created by: Gilbert
###################################
def GetMinMax(data, region = 'VIC', buy_threshold = 5, sell_threshold = 4):
    if region == 'VIC':
        spot_price = data['Regions VIC Trading Price ($/MWh)']
    elif region == 'NSW':
        spot_price = data['Regions NSW Trading Price ($/MWh)']
    elif region == 'SA':
        spot_price = data['Regions SA Trading Price ($/MWh)']
    elif region == 'TAS':
        spot_price = data['Regions TAS Trading Price ($/MWh)']
    
    price = np.array(spot_price)
    minimum_price = np.argsort(price, kind = 'merge*sort') # (O(NlogN)), mergesort the minimum prices.
    maximum_price = minimum_price[::-1][:len(price)] # (O(N)), maximum is the reverse order of minimum.
    
    selected_min_price = [EMPTY for i in minimum_price] # (O(N)), set an empty array for the whole period.
    selected_max_price = [EMPTY for i in minimum_price] # (O(N)), set an empty array for the whole period.
    
    # Select the lowest price spot over the given
    # buy_threshold as the minimum buying point.
    i = 0
    for b_t in range(buy_threshold):
        selected_min_price[minimum_price[i]] = b_t + 1
        i += 1
        
    # Select the highest price spot over the given
    # sell_threshold as the maximum selling point.
    i = 0
    for s_t in range(sell_threshold):
        selected_max_price[maximum_price[i]] = s_t + 1
        i += 1
        
    return selected_min_price, selected_max_price

########################################################################################

###################################
# A function to Find Battery Charge and Discharge pairs in backward order.
# Backward order from 48th period to the 1st.
#
# Parameters:
#      - buy_period : Selected minimum price point as it will be where we buy energy for charging.
#      - sell_period : Selected maximum price point as it will be where we sell energy for discharging.
#
# Return:
#      - List of battery class pairs
#
# Efficiency: O(N)
#
# Created by: Gilbert
###################################
def FindBatteryPairs(buy_period, sell_period):
    MAX_SELL_PERIOD = 4 # MAXIMUM SELLING PERIOD PER PAIR
    MAX_BUY_PERIOD = 5 # MAXIMUM BUYING PERIOD PER PAIR
    
    period = len(buy_period)
    
    battery = []
    sell = OrderedDict() # Initialise battery selling point. (Ordered Dictionary)
    buy = OrderedDict() # Initialise battery buying point. (Orderered Dictionary)
    
    # Iterate over the whole period backwards
    for p in range(period - 1, -1, -1):
        # If maximum selling point is not empty, add (order, period)
        # as key-value pair into the OrderedDict.
        if sell_period[p] != EMPTY and buy_period[p] == EMPTY:
            # If battery buying point period is less MAXIMUM SELLING 
            # PERIOD PER PAIR, add new period.
            if len(sell) < MAX_SELL_PERIOD:
                sell[sell_period[p]] = sell_period.index(sell_period[p]) + 1
            # else, if battery selling point is full and there is 
            # higher maximum selling point then remove the lowest
            # selling point and add the new one into Dictionary.    
            else:
                max_key = max(sell, key=int)
                if sell_period[p] < max_key:
                    sell.pop(max_key)
                    sell[sell_period[p]] = sell_period.index(sell_period[p]) + 1

        # If battery selling point is not empty and minimum buying 
        # point is not empty.
        if len(sell) != 0 and sell_period[p] == EMPTY and buy_period[p] != EMPTY :
            # If battery buying point period is less MAXIMUM BUYING 
            # PERIOD PER PAIR, add new period.
            if len(buy) < MAX_BUY_PERIOD and len(buy) < math.ceil(len(sell) * 1.25):
                buy[buy_period[p]] = buy_period.index(buy_period[p]) + 1
            # else, if battery buying point is full and there is 
            # lower minimum buying point then remove the highest
            # buying point and add the new one into Dictionary.
            else:
                max_key = max(buy, key=int)
                if buy_period[p] < max_key:
                    buy.pop(max_key)
                    buy[buy_period[p]] = buy_period.index(buy_period[p]) + 1
        # If the next period is not empty and battery buying point
        # is not empty then battery charge-discharge pair has been
        # created.
        # Reinitialise a new battery setup.
        if sell_period[p - 1] != EMPTY and len(buy) != 0:
            battery.append([list(sell.items()), list(buy.items())])
            sell = OrderedDict()
            buy = OrderedDict()
    # Add the last battery charge-discharge pair occuring 
    # before 1st period.
    battery.append([list(sell.items()), list(buy.items())]) 

    # Check whether there is too many selling points, then
    # remove selling point until the number of selling points
    # is equal to the number of buying points while removing
    # the lowest selling point.
    for b in battery:
        sell_tmp = np.array(b[0])
        buy_tmp = b[1]
        while len(sell_tmp) > len(buy_tmp): 
            row = 0
            index = np.where(sell_tmp[:,0] == sell_tmp[:,0].max())[0][0]
            sell_tmp = np.delete(sell_tmp, index, axis = row)
        b[0] = sell_tmp.tolist() # Change numpy array to list

    return battery

########################################################################################

###################################
# A function to set optimal charge and discharge amount of battery pairs.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - battery_pairs : list of all battery class pairs.
#
# Return:
#      - List of all battery class pairs
#
# Efficiency: O(N)
#
# Created by: Gilbert
###################################
def SetChargeDischarge(data, battery_pairs, region = "VIC"):    
    all_batteries = []
    
    battery_pairs = battery_pairs[::-1]
    for b in battery_pairs:
        sell_period = b[0][::-1] # Reverse the order
        buy_period = b[1][::-1] # Reverse the order
        if len(sell_period)!= 0 or len(buy_period) != 0:
            sell_price = GetSpotPrice(data, sell_period, region = region)
            buy_price = GetSpotPrice(data, buy_period, region = region)

            battery = Battery(buy_period, buy_price, sell_period, sell_price)
            battery.Setting()
            battery.FirstOptimisation()
            battery.SecondOptimisation()

            all_batteries.append(battery)

    return all_batteries

########################################################################################

###################################
# A function to calculate daily revenue.
#
# Parameters:
#      - all_batteries : List of battery class pairs.
#
# Return:
#      - Daily revenues
#
# Created by: Gilbert
###################################
def ComputeDailyRevenue(all_batteries):
    revenues = 0
    for battery in all_batteries:
        revenues += battery.Revenue()
    return revenues

########################################################################################

###################################
# A function to optimise battery charging and discharging period. This is where 
# mainly the optimisations are performed with the helper functions.
#
# Parameters:
#      - daily_data : the targeted daily dataset, minimum dataset length of 48.
#
# Return:
#      - raw_power : List of Daily Raw Power for charging and discharging
#      - market_dispatch : List of Daily Market Dispatch for charging and discharging
#      - opening_capacity : List of Daily Opening Capacity for charging and discharging
#      - closing_capacity : List of Daily Closing Capacity for charging and discharging
#
# Efficiency: O(N^2 * NLogN) = O(N^3LogN)
# Created by: Gilbert
###################################
def PeriodOptimisation(given_data, region = "VIC"):
    best_batteries = OrderedDict() # Initialise an Ordered Dictionary
    
    # TODO: OPTIMISE EFFICIENCY HERE! Make this at least < N^2
    # Iterate over all possible combinations of battery pairs. (O(N^2))
    period = len(given_data)
    count = 0
    for s in range(1, period + 1):
        for b in range(1 , period + 1 - s): # change this to range(1, period - s + 1) to reduce by half. 
            # Get the minimum and maximum price based on the given threshold
            min_price, max_price = GetMinMax(given_data, buy_threshold = b, sell_threshold = s, region = region)
            # Get the battery pairs based on minimum and maximum price
            battery_pairs = FindBatteryPairs(min_price, max_price)
            # Get battery optimisation for the selected threshold
            all_batteries = SetChargeDischarge(given_data, battery_pairs, region = region)
            # Compute daily revenues of selected battery combinations
            dailyrev = ComputeDailyRevenue(all_batteries)
            #print((b, s), dailyrev)
            if dailyrev < 0:
                break
            count += 1
            # Insert revenue as key, batteries combination and threshold as value
            if dailyrev not in best_batteries:
                best_batteries[dailyrev] = (all_batteries, (b, s))
                
    # Find the highest revenue amongst possible combinations in that day
    best_revenue = max(best_batteries)
    #best_threshold = best_batteries[best_revenue][1]
    #print(best_threshold)
    battery = best_batteries[best_revenue][0] # The Best battery combinations
    
    # Initialise raw_power, market_dispatch, opening_capacity and closing capacity
    raw_power = [0 for i in range(period)]
    market_dispatch = [0 for i in range(period)]
    opening_capacity = [0 for i in range(period)]
    closing_capacity = [0 for i in range(period)]
    
    # Iterate over battery combinations to set raw_power, market_dispatch,
    # opening_capacity, closing_capacity into an array to be prepared for 
    # merging with the dataset.
    for b in battery:   
        # Charging Period
        for cp in range(len(b.charge_period)):
            raw_power[b.charge_period[cp][1] - 1] = b.charge_raw_power[cp]
            market_dispatch[b.charge_period[cp][1] - 1] = b.charge_market_dispatch[cp]
            opening_capacity[b.charge_period[cp][1] - 1] = b.charge_capacity[cp][0]
            closing_capacity[b.charge_period[cp][1] - 1] = b.charge_capacity[cp][1]
        # Discharge Period
        for dp in range(len(b.discharge_period)):
            raw_power[b.discharge_period[dp][1] - 1] = b.discharge_raw_power[dp]
            market_dispatch[b.discharge_period[dp][1] - 1] = b.discharge_market_dispatch[dp]
            opening_capacity[b.discharge_period[dp][1] - 1] = b.discharge_capacity[dp][0]
            closing_capacity[b.discharge_period[dp][1] - 1] = b.discharge_capacity[dp][1]
    
    # Formatting the opening and closing capacity.
    for i in range(1, len(opening_capacity)):
        if closing_capacity[i - 1] != 0.0 and opening_capacity[i] == 0.0:
            opening_capacity[i] = closing_capacity[i - 1]
            closing_capacity[i] = opening_capacity[i]
                
    return raw_power, market_dispatch, opening_capacity, closing_capacity, best_revenue

def FixedOptimisation(data, period = 48, region = 'VIC'):
    raw_power = []
    market_dispatch = []
    opening_capacity = []
    closing_capacity = []

    start = 0
    end = period 

    while end <= len(data):
        tmp_data = data.iloc[start:end, :]
        daily_rp, daily_md, daily_oc, daily_cc, _ = PeriodOptimisation(tmp_data)
        raw_power.extend(daily_rp)
        market_dispatch.extend(daily_md)
        opening_capacity.extend(daily_oc)
        closing_capacity.extend(daily_cc)
        print(end)
        start += period
        end += period

    data['Raw Power (MW)'] = pd.Series(raw_power)
    data['Market Dispatch (MWh)'] = pd.Series(market_dispatch)
    data['Opening Capacity (MWh)'] = pd.Series(opening_capacity)
    data['Closing Capacity (MWh)'] = pd.Series(closing_capacity)
    
    return data

### Check Dependency

In [4]:
def renameColumns(dataframe, region = 'VIC'):
    if region == 'VIC':
        price_name = 'Regions VIC Trading Price ($/MWh)'
    elif region == 'NSW':
        price_name = 'Regions NSW Trading Price ($/MWh)'
    elif region == 'SA':
        price_name = 'Regions SA Trading Price ($/MWh)'
    elif region == 'TAS':
        price_name = 'Regions TAS Trading Price ($/MWh)'
        
    dataframe.columns = ['Time (UTC+10)', price_name, 'Raw Power (MW)', 'Market Dispatch (MWh)', 'Opening Capacity (MWh)', 'Closing Capacity (MWh)', 'Revenue ($)']
    return dataframe

def computeRevenue(dataframe):
    return dataframe['Revenue ($)'].sum()

In [78]:
PERIOD = 48

def getDependency(dataframe):
    dayBorder = []
    
    dependency = []
    for i in range(PERIOD, len(dataframe), PERIOD):
        dayBorder.append((i - 1, i))
        if dataframe.loc[i - 1, 'Closing Capacity (MWh)'] != 0 and dataframe.loc[i, 'Opening Capacity (MWh)'] != 0:
            dependency.append((i - 1, i))
            
    consecutiveDependency = []
    tmp = []
    for d in range(len(dependency) - 1):
        curr_d = dependency[d][0]
        next_d = dependency[d + 1][0]
        tmp.append(dependency[d])
        if (next_d - curr_d != PERIOD):
            consecutiveDependency.append(tmp)
            tmp = []
       
    if dependency[-1][0] - consecutiveDependency[-1][0][0] == PERIOD:
        consecutiveDependency[-1].append(dependency[-1])
    else:
        consecutiveDependency.append([dependency[-1]])

    for c in range(len(consecutiveDependency)):
        current = consecutiveDependency[c]
        if len(current) != 1:
            consecutiveDependency[c] = [current[0], current[-1]]
            
    return consecutiveDependency

def createPeriod(current, until):
    tmp = []
    for i in range(current, until, PERIOD):
        tmp.append((i, i + PERIOD - 1))
    return tmp

def getTimePeriod(dataframe):
    dependency = getDependency(dataframe)
    
    tmp = []
    for depend in dependency:
        openPeriod = depend[0][0] - PERIOD + 1     # 48th period - 47 = 1st period
        closePeriod = depend[-1][-1] + PERIOD - 1  # 1st period + 47 = 48th period
        tmp.append((openPeriod, closePeriod))
        
    timeIndex = []
    timeIndex.extend(createPeriod(0, tmp[0][0]))
    
    for t in range(1, len(tmp)):
        timeIndex.append((tmp[t-1]))
        timeIndex.extend(createPeriod(tmp[t-1][1] + 1, tmp[t][0]))
    timeIndex.append(tmp[t])  
    
    timePeriod = []
    for index in timeIndex:
        timePeriod.append((dataframe.iloc[index[0], 0], dataframe.iloc[index[1], 0], index[1] - index[0] + 1))
   
    return timePeriod

In [70]:
def DependencyOptimisation(data, timePeriod, region = "VIC"):
    raw_power = []
    market_dispatch = []
    opening_capacity = []
    closing_capacity = []
    
    count = 0
    for time in timePeriod:
        start_t = time[0]
        end_t = time[1]
        period_t = time[2]
        
        data_interval = data.loc[(data['Time (UTC+10)'] >= start_t) & \
                                 (data['Time (UTC+10)'] <= end_t)]
        
        original_revenue = computeRevenue(data_interval)
        
        tmp_rev = 0
        if period_t > PERIOD:
            start = 0
            end = PERIOD 
            tmp_rp = []
            tmp_md = []
            tmp_oc = []
            tmp_cc = []
            while end <= len(data_interval):
                tmp_data = data_interval.iloc[start:end, :]
                daily_rp, daily_md, daily_oc, daily_cc, revenue_1 = PeriodOptimisation(tmp_data, region)
                tmp_rev += revenue_1
                tmp_rp.extend(daily_rp)
                tmp_md.extend(daily_md)
                tmp_oc.extend(daily_oc)
                tmp_cc.extend(daily_cc)
                
                start += PERIOD
                end += PERIOD
            
        daily_rp, daily_md, daily_oc, daily_cc, revenue_2 = PeriodOptimisation(data_interval, region)
        if (revenue_2 > original_revenue or tmp_rev > original_revenue):
            if tmp_rev != 0 and tmp_rev > revenue_2:
                raw_power.extend(tmp_rp)
                market_dispatch.extend(tmp_md)
                opening_capacity.extend(tmp_oc)
                closing_capacity.extend(tmp_cc)
            else:
                raw_power.extend(daily_rp)
                market_dispatch.extend(daily_md)
                opening_capacity.extend(daily_oc)
                closing_capacity.extend(daily_cc)
        else:
            raw_power.extend(data_interval['Raw Power (MW)'])
            market_dispatch.extend(data_interval['Market Dispatch (MWh)'])
            opening_capacity.extend(data_interval['Opening Capacity (MWh)'])
            closing_capacity.extend(data_interval['Closing Capacity (MWh)'])
            
        print(count)
        count += 1
    
    data['Raw Power (MW)'] = pd.Series(raw_power)
    data['Market Dispatch (MWh)'] = pd.Series(market_dispatch)
    data['Opening Capacity (MWh)'] = pd.Series(opening_capacity)
    data['Closing Capacity (MWh)'] = pd.Series(closing_capacity)
    
    return data

In [93]:
data_SM = pd.read_excel("../../preprocessed_data/First Algorithm/SM.xlsx")
data_SM = data_SM[['Time', 'Price', 'Raw Power', 'Actual', 'Opening Capacity', 'Closing Capacity', 'Revenue']]
data_SM = renameColumns(data_SM)
timePeriod = getTimePeriod(data_SM)
for t in timePeriod:
    print(t)

(Timestamp('2018-01-01 00:30:00'), Timestamp('2018-01-02 00:00:00'), 48)
(Timestamp('2018-01-02 00:30:00'), Timestamp('2018-01-03 00:00:00'), 48)
(Timestamp('2018-01-03 00:30:00'), Timestamp('2018-01-04 00:00:00'), 48)
(Timestamp('2018-01-04 00:30:00'), Timestamp('2018-01-06 00:00:00'), 96)
(Timestamp('2018-01-06 00:30:00'), Timestamp('2018-01-07 00:00:00'), 48)
(Timestamp('2018-01-07 00:30:00'), Timestamp('2018-01-09 00:00:00'), 96)
(Timestamp('2018-01-09 00:30:00'), Timestamp('2018-01-10 00:00:00'), 48)
(Timestamp('2018-01-10 00:30:00'), Timestamp('2018-01-12 00:00:00'), 96)
(Timestamp('2018-01-12 00:30:00'), Timestamp('2018-01-13 00:00:00'), 48)
(Timestamp('2018-01-13 00:30:00'), Timestamp('2018-01-14 00:00:00'), 48)
(Timestamp('2018-01-14 00:30:00'), Timestamp('2018-01-16 00:00:00'), 96)
(Timestamp('2018-01-16 00:30:00'), Timestamp('2018-01-18 00:00:00'), 96)
(Timestamp('2018-01-18 00:30:00'), Timestamp('2018-01-19 00:00:00'), 48)
(Timestamp('2018-01-19 00:30:00'), Timestamp('2018-

In [94]:
example = DependencyOptimisation(data_SM, timePeriod)
example.to_excel('../../preprocessed_data/First Algorithm/example_SM.xlsx', index = False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [96]:
test = pd.read_excel('../../preprocessed_data/First Algorithm/example_SM.xlsx')
test[test['Regions VIC Trading Price ($/MWh)'] > 1000]

,Time (UTC+10),Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh),Revenue ($),Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
845,2018-01-18 15:00:00,1775.20,300.0,135.0,580,430,2.374951e+05,0.000000,300,0,NaN,NaN
847,2018-01-18 16:00:00,1960.54,0.0,0.0,565,565,0.000000e+00,-33.333333,300,0,NaN,NaN
848,2018-01-18 16:30:00,5682.01,300.0,135.0,565,415,7.601677e+05,-33.333333,300,0,NaN,NaN
849,2018-01-18 17:00:00,12931.04,300.0,135.0,415,265,1.729979e+06,-300.000000,300,0,NaN,NaN
850,2018-01-18 17:30:00,11960.16,300.0,135.0,265,115,1.600090e+06,-300.000000,300,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
59796,2021-05-30 18:30:00,2224.28,300.0,135.0,430,280,2.975753e+05,-300.000000,300,0,NaN,NaN
59797,2021-05-30 19:00:00,1703.36,300.0,135.0,280,130,2.278840e+05,-300.000000,300,0,NaN,NaN
60850,2021-06-21 17:30:00,2642.65,300.0,135.0,430,280,3.535469e+05,-300.000000,300,0,NaN,NaN
60851,2021-06-21 18:00:00,1876.67,300.0,135.0,280,130,2.510703e+05,-300.000000,300,0,NaN,NaN


## Development

In [97]:
data_128 = pd.read_excel("../../preprocessed_data/First Algorithm/128.xlsx")
data_128 = data_128[['Time', 'Price', 'Raw Power', 'Actual', 'Opening Capacity', 'Closing Capacity', 'Revenue']]
data_128

,Time,Price,Raw Power,Actual,Opening Capacity,Closing Capacity,Revenue
0,2018-01-01 00:30:00,92.46,0.0,0.0,0.0,0.0,0.000000
1,2018-01-01 01:00:00,87.62,0.0,0.0,0.0,0.0,0.000000
2,2018-01-01 01:30:00,73.08,0.0,0.0,0.0,0.0,0.000000
3,2018-01-01 02:00:00,70.18,0.0,0.0,0.0,0.0,0.000000
4,2018-01-01 02:30:00,67.43,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...
63451,2021-08-14 22:00:00,49.93,-300.0,-150.0,0.0,135.0,-7557.517659
63452,2021-08-14 22:30:00,62.86,270.0,121.5,135.0,0.0,7568.752590
63453,2021-08-14 23:00:00,32.26,0.0,0.0,0.0,0.0,0.000000
63454,2021-08-14 23:30:00,25.10,0.0,0.0,0.0,0.0,0.000000


In [98]:
data_128 = renameColumns(data_128)
timePeriod = getTimePeriod(data_128)

In [155]:
def FindBatteryPairsV2(buy_period, sell_period):
    MAX_SELL_PERIOD = 4 # MAXIMUM SELLING PERIOD PER PAIR
    MAX_BUY_PERIOD = 5 # MAXIMUM BUYING PERIOD PER PAIR
    
    period = len(buy_period)
    
    battery = []
    sell = OrderedDict() # Initialise battery selling point. (Ordered Dictionary)
    buy = OrderedDict() # Initialise battery buying point. (Orderered Dictionary)
    
    #for p in range(period - 1, -1, -1):
        

    return 
    
###################################
# A function to find local minima and maxima of a given period given the data.
# A local minima can be defined as when the adjacent (previous and next) elements are
# greater than the current value. 
# Similarly, a local Maxima can be defined when the adjacent elements are smaller than
# the current value.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48
#      - period (default = 48) : preferable period with minimum of 48.
#
# Return:
#      - list of local minima, list of local maxima
#
# Created by: Gilbert
# Referenced from: https://www.geeksforgeeks.org/find-indices-of-all-local-maxima-and-local-minima-in-an-array/
###################################

def LocalMinMax(data, period = 48):
    # Set number of maximum discharge
    max_discharge_number = 5 # (5 * 5 = 25)
    max_charge_number = 5 # (5 * 4 = 20)
     
    len_data = len(data)
    
    local_max_id = [] 
    local_min_id = [] 
    
    # Change dataframe into numpy array
    spot_price = data['Regions VIC Trading Price ($/MWh)'].to_numpy()
    
    # Check the first element if it is local minima or maxima.
    # If first element is greater than the next, then it is 
    # local maxima and vice versa.
    if spot_price[0] > spot_price[1]:
        local_max_id.append(0)
    elif spot_price[0] < spot_price[1]:
        local_min_id.append(0)
    
    # Iterating over the second to the second to last element,
    # checking which element is local minima or maxima
    for i in range(1, len_data - 1):
        if(spot_price[i - 1] > spot_price[i] < spot_price[i + 1]): 
            local_min_id.append(i) 
        elif(spot_price[i-1] < spot_price[i] > spot_price[i + 1]): 
            local_max_id.append(i) 
    
    # Check the last element if it is local minima or maxima. 
    # If last element is greater than second to last, then it 
    # is local maxima and vice versa.
    if(spot_price[-1] > spot_price[-2]): 
        local_max_id.append(len_data - 1) 
    elif(spot_price[-1] < spot_price[-2]): 
        local_min_id.append(len_data - 1) 
    
    # Create a sorted dictionary of local maxima and minima
    local_max = {}
    local_min = {}
    for m in local_max_id:
        local_max[m] = spot_price[m]
    for m in local_min_id:
        local_min[m] = spot_price[m]
    
    sorted_local_max = sorted(local_max.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)
    sorted_local_min = sorted(local_min.items(), key = lambda kv:(kv[1], kv[0]))
    
    # Initialise empty array
    period_maxima = np.empty(len_data)
    period_maxima[:] = np.NaN
    
    period_minima = np.empty(len_data)
    period_minima[:] = np.NaN
    
    mod = len_data // period
    # Set the given local maxima index to a set length of period.
    # The maximum number of maxima set is 4 * (length of data // period).
    # For example, it can be assumed that if battery charge and discharge
    # is 1:1 ratio with a period of 48 and the time taken to charge is 5 
    # period and discharge is 4 period. Therefore, the possible maximum 
    # is 45 period in a day which are 5 times for each charge (5 * 5) and 
    # discharge (5 * 4) 

    for p_max in range(len(sorted_local_max)):
        period_maxima[sorted_local_max[p_max][0]] = p_max + 1
        #if p_max + 1 == max_discharge_number * mod:
        #    break
        
    for p_min in range(len(sorted_local_min)):
        period_minima[sorted_local_min[p_min][0]] = p_min + 1
        #if p_min + 1 == max_charge_number * mod:
        #    break
            
    return period_minima, period_maxima

In [146]:
data_interval = data_128.loc[(data_128['Time (UTC+10)'] >= '2018-01-18 00:30:00') & \
                        (data_128['Time (UTC+10)'] <= '2018-01-19 00:00:00')].copy()
data_interval = data_interval.reset_index(drop = True)
data_interval

,Time (UTC+10),Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh),Revenue ($)
0,2018-01-18 00:30:00,66.33,0.000000,0.000000,0.0,0.0,0.000000e+00
1,2018-01-18 01:00:00,64.18,0.000000,0.000000,0.0,0.0,0.000000e+00
2,2018-01-18 01:30:00,63.07,0.000000,0.000000,0.0,0.0,0.000000e+00
3,2018-01-18 02:00:00,67.56,0.000000,0.000000,0.0,0.0,0.000000e+00
4,2018-01-18 02:30:00,65.95,0.000000,0.000000,0.0,0.0,0.000000e+00
5,2018-01-18 03:00:00,63.38,0.000000,0.000000,0.0,0.0,0.000000e+00
6,2018-01-18 03:30:00,62.60,0.000000,0.000000,0.0,0.0,0.000000e+00
7,2018-01-18 04:00:00,61.87,-300.000000,-150.000000,0.0,135.0,-9.364783e+03
8,2018-01-18 04:30:00,59.69,-300.000000,-150.000000,135.0,270.0,-9.034813e+03
9,2018-01-18 05:00:00,66.01,0.000000,0.000000,270.0,270.0,0.000000e+00


In [129]:
FixedOptimisation(data_interval, period=len(data_interval))

9


,Time (UTC+10),Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh),Revenue ($)
0,2018-01-18 20:00:00,124.28,0.000000,0.000000,0.0,0.0,0.000000
1,2018-01-18 20:30:00,103.98,-300.000000,-150.000000,0.0,135.0,-15738.647830
2,2018-01-18 21:00:00,104.00,-33.333333,-16.666667,135.0,150.0,0.000000
3,2018-01-18 21:30:00,139.75,300.000000,135.000000,150.0,0.0,16826.808375
4,2018-01-18 22:00:00,82.73,-300.000000,-150.000000,0.0,135.0,-12522.199798
5,2018-01-18 22:30:00,117.27,0.000000,0.000000,135.0,135.0,0.000000
6,2018-01-18 23:00:00,114.38,-33.333333,-16.666667,135.0,150.0,0.000000
7,2018-01-18 23:30:00,147.70,300.000000,135.000000,150.0,0.0,17784.040050
8,2018-01-19 00:00:00,118.69,0.000000,0.000000,0.0,0.0,0.000000


In [156]:
min_price, max_price = GetMinMax(data_interval, buy_threshold = 37, sell_threshold = 11)
minima, maxima = LocalMinMax(data_interval)
for i in range(len(min_price)):
    print(f"{data_interval.loc[i, 'Regions VIC Trading Price ($/MWh)']}: {min_price[i]}, {max_price[i]}, {minima[i]}, {maxima[i]}")
#FindBatteryPairsV2(min_price, max_price)

66.33: 13,  , nan, 11.0
64.18: 10,  , nan, nan
63.07: 8,  , 4.0, nan
67.56: 14,  , nan, 10.0
65.95: 11,  , nan, nan
63.38: 9,  , nan, nan
62.6: 6,  , nan, nan
61.87: 4,  , nan, nan
59.69: 1,  , 1.0, nan
66.01: 12,  , nan, nan
70.14: 15,  , nan, 9.0
60.72: 3,  , nan, nan
60.43: 2,  , 2.0, nan
63.04: 7,  , nan, 12.0
62.3: 5,  , 3.0, nan
79.55: 19,  , nan, nan
95.07: 26,  , nan, nan
110.51: 32,  , nan, 7.0
73.11: 16,  , 5.0, nan
74.89: 17,  , nan, nan
75.46: 18,  , nan, nan
84.73: 21,  , nan, nan
87.91: 23,  , nan, nan
94.21: 25,  , nan, nan
100.81: 27,  , nan, 8.0
93.26: 24,  , 8.0, nan
105.53: 30,  , nan, nan
107.89: 31,  , nan, nan
136.05: 37,  , nan, nan
1775.2:  , 6, nan, 2.0
86.72: 22,  , 7.0, nan
1960.54:  , 5, nan, nan
5682.01:  , 3, nan, nan
12931.04:  , 1, nan, 1.0
11960.16:  , 2, nan, nan
5078.6:  , 4, nan, nan
238.45:  , 7, nan, nan
178.84:  , 9, 12.0, nan
230.07:  , 8, nan, 3.0
124.28: 36,  , nan, nan
103.98: 28,  , 9.0, nan
104.0: 29,  , nan, nan
139.75:  , 11, nan, 5.0
82.7

In [90]:
example_128 = DependencyOptimisation(data_128, timePeriod)
example_128.to_excel('../../preprocessed_data/First Algorithm/example_128_V3.xlsx', index = False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [91]:
test = pd.read_excel('../../preprocessed_data/First Algorithm/example_128_V3.xlsx')
test[test["Opening Capacity (MWh)"] != test["Closing Capacity (MWh)"].shift()]

,Time (UTC+10),Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh),Revenue ($)
0,2018-01-01 00:30:00,92.46,0.0,0.0,0.0,0.0,0.0


## Original Data

In [7]:
market_data = pd.read_excel("../../raw_data/market_data.xlsx")
market_data.drop(index=market_data.index[0], axis=0, inplace=True)
market_data = market_data.reset_index(drop = True)

In [8]:
market_data

,Time (UTC+10),Regions NSW Trading Price ($/MWh),Regions SA Trading Price ($/MWh),Regions TAS Trading Price ($/MWh),Regions VIC Trading Price ($/MWh),Regions NSW Trading Total Intermittent Generation (MW),Regions SA Trading Total Intermittent Generation (MW),Regions TAS Trading Total Intermittent Generation (MW),Regions VIC Trading Total Intermittent Generation (MW),Regions NSW Operational Demand (MW),Regions SA Operational Demand (MW),Regions TAS Operational Demand (MW),Regions VIC Operational Demand (MW)
0,2018-01-01 00:30:00,91.86,107.17,92.28,92.46,0.15,43.07,118.73,131.68,6974,1359,1082,4398
1,2018-01-01 01:00:00,88.83,103.31,87.53,87.62,0.13,41.67,110.48,119.98,6790,1316,1071,4238
2,2018-01-01 01:30:00,73.62,88.20,76.29,73.08,0.14,42.15,120.09,123.86,6536,1240,1067,4112
3,2018-01-01 02:00:00,71.49,85.24,75.10,70.18,0.16,38.31,114.64,132.72,6339,1194,1061,3956
4,2018-01-01 02:30:00,69.27,81.75,72.92,67.43,0.16,33.39,112.90,120.73,6195,1163,1060,3833
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63451,2021-08-14 22:00:00,50.84,59.16,7.54,49.93,8.74,36.42,154.69,182.52,8491,1718,1255,5492
63452,2021-08-14 22:30:00,66.85,80.01,10.52,62.86,7.52,51.83,156.09,206.23,8376,1665,1244,5344
63453,2021-08-14 23:00:00,55.64,77.76,7.63,32.26,7.69,42.04,160.30,215.20,8194,1614,1207,5204
63454,2021-08-14 23:30:00,52.25,76.47,7.52,25.10,8.35,38.04,167.00,226.95,8022,1573,1163,5268


### Add Period

In [9]:
# Create a period for a whole day which are 48 as
# Spot prices are taken by the 30 minutes mark.
period = []
count = 1
for i in range(1, len(market_data) + 1):
    period.append(count)
    count += 1
    if (i % 48) == 0:
        count = 1
        
market_data['Period'] = pd.Series(period)
market_data['Time (UTC+10)'] = pd.to_datetime(market_data['Time (UTC+10)'])

In [10]:
vic_spot_price = market_data[["Time (UTC+10)", "Period", "Regions VIC Trading Price ($/MWh)"]].copy()
vic_spot_price

,Time (UTC+10),Period,Regions VIC Trading Price ($/MWh)
0,2018-01-01 00:30:00,1,92.46
1,2018-01-01 01:00:00,2,87.62
2,2018-01-01 01:30:00,3,73.08
3,2018-01-01 02:00:00,4,70.18
4,2018-01-01 02:30:00,5,67.43
...,...,...,...
63451,2021-08-14 22:00:00,44,49.93
63452,2021-08-14 22:30:00,45,62.86
63453,2021-08-14 23:00:00,46,32.26
63454,2021-08-14 23:30:00,47,25.10


In [31]:
noDepend = FixedOptimisation(vic_spot_price)
noDepend.to_excel('../../preprocessed_data/First Algorithm/example_fixedOpt.xlsx', index = False)

48
96
144
192
240
288
336
384
432
480
528
576
624
672
720
768
816
864
912
960
1008
1056
1104
1152
1200
1248
1296
1344
1392
1440
1488
1536
1584
1632
1680
1728
1776
1824
1872
1920
1968
2016
2064
2112
2160
2208
2256
2304
2352
2400
2448
2496
2544
2592
2640
2688
2736
2784
2832
2880
2928
2976
3024
3072
3120
3168
3216
3264
3312
3360
3408
3456
3504
3552
3600
3648
3696
3744
3792
3840
3888
3936
3984
4032
4080
4128
4176
4224
4272
4320
4368
4416
4464
4512
4560
4608
4656
4704
4752
4800
4848
4896
4944
4992
5040
5088
5136
5184
5232
5280
5328
5376
5424
5472
5520
5568
5616
5664
5712
5760
5808
5856
5904
5952
6000
6048
6096
6144
6192
6240
6288
6336
6384
6432
6480
6528
6576
6624
6672
6720
6768
6816
6864
6912
6960
7008
7056
7104
7152
7200
7248
7296
7344
7392
7440
7488
7536
7584
7632
7680
7728
7776
7824
7872
7920
7968
8016
8064
8112
8160
8208
8256
8304
8352
8400
8448
8496
8544
8592
8640
8688
8736
8784
8832
8880
8928
8976
9024
9072
9120
9168
9216
9264
9312
9360
9408
9456
9504
9552
9600
9648
9696
9744
9792
98

In [92]:
test = pd.read_excel('../../preprocessed_data/First Algorithm/Dependency_ON_SM.xlsx')
test[test["Opening Capacity (MWh)"] != test["Closing Capacity (MWh)"].shift()]

,Time (UTC+10),Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh),Revenue ($),Max Charge,Max Discharge,Flags,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,2018-01-01 00:30:00,92.46,0.0,0.0,0,0,0.000000,-300.0,0,0,NaN,NaN,NaN
3648,2018-03-18 00:30:00,130.95,270.0,121.5,135,0,15767.231175,-300.0,270,0,NaN,NaN,NaN
3936,2018-03-24 00:30:00,93.18,210.0,94.5,105,0,8726.260410,-300.0,210,0,NaN,NaN,NaN
5616,2018-04-28 00:30:00,85.82,260.0,117.0,130,0,9950.571540,-300.0,260,0,NaN,NaN,NaN
6000,2018-05-06 00:30:00,87.76,300.0,135.0,150,0,11740.971600,-300.0,300,0,NaN,NaN,NaN
8736,2018-07-02 00:30:00,125.03,300.0,135.0,150,0,16727.138550,-300.0,300,0,NaN,NaN,NaN
8832,2018-07-04 00:30:00,43.36,270.0,121.5,135,0,5220.825840,-300.0,270,0,NaN,NaN,NaN
12816,2018-09-25 00:30:00,127.95,240.0,108.0,120,0,13694.232600,-300.0,240,0,NaN,NaN,NaN
13056,2018-09-30 00:30:00,268.64,300.0,135.0,150,0,35940.002400,-300.0,300,0,NaN,NaN,NaN
14400,2018-10-28 00:30:00,173.25,300.0,135.0,150,0,23178.251250,-300.0,300,0,NaN,NaN,NaN


---